In [ ]:
class SATSolver:
    def __init__(self, clauses):
        self.clauses = clauses
        self.assignment = {}

    def solve(self):
        while True:
            # Propagate unary clauses
            while True:
                unary_clauses = [clause[0] for clause in self.clauses if len(clause) == 1]
                if not unary_clauses:
                    break
                literal = unary_clauses[0]
                self.assign_literal(literal)

            # Check for conflicting clauses
            conflicting_clause = self.get_conflicting_clause()
            if conflicting_clause is not None:
                if not self.backtrack():
                    return False
            elif len(self.assignment) == len(self.clauses):
                return True
            else:
                # Select a literal for branching
                literal = self.select_literal()
                self.assign_literal(literal, branch=True)

    def assign_literal(self, literal, branch=False):
        variable = abs(literal)
        self.assignment[variable] = literal
        if branch:
            self.clauses.append([literal])

        # Remove clauses containing the assigned literal
        self.clauses = [clause for clause in self.clauses if variable not in map(abs, clause)]

        # Remove negated literal from remaining clauses
        negated_literal = -literal
        self.clauses = [[l for l in clause if l != negated_literal] for clause in self.clauses]

    def get_conflicting_clause(self):
        for clause in self.clauses:
            literals = set(clause)
            if all(-l not in literals for l in clause):
                return clause
        return None

    def backtrack(self):
        conflicting_clause = self.get_conflicting_clause()
        while conflicting_clause is not None:
            # Conflict resolution by learning a new clause
            learned_clause = [-l for l in conflicting_clause]
            self.clauses.append(learned_clause)
            conflicting_clause = self.get_conflicting_clause()

            # Backtrack to the previous decision level
            backtrack_level = self.get_backtrack_level()
            if backtrack_level is None:
                return False
            self.undo_assignments(backtrack_level)

        return True

    def get_backtrack_level(self):
        if not self.assignment:
            return None

        max_level = max([abs(self.assignment[variable]) for variable in self.assignment])
        return max_level if max_level > 0 else None


    def undo_assignments(self, backtrack_level):
        variables_to_remove = [variable for variable in self.assignment if abs(self.assignment[variable]) > backtrack_level]
        for variable in variables_to_remove:
            del self.assignment[variable]

    def select_literal(self):
        # Select the first literal from the shortest clause
        self.clauses.sort(key=lambda clause: len(clause))
        return self.clauses[0][0]


# Example usage
clauses = [[1, -2, 3], [2, 5]]
solver = SATSolver(clauses)
if solver.solve():
    print("Satisfiable")
    for variable, literal in solver.assignment.items():
        print(f"Variable {variable}: {literal}")
else:
    print("Unsatisfiable")
